In [ ]:
from leann.api import LeannBuilder, LeannSearcher, LeannChat
# 1. Build index (no embeddings stored!)
builder = LeannBuilder(backend_name="hnsw")
builder.add_text("C# is a powerful programming language but it is not very popular")
builder.add_text("Python is a powerful programming language and it is very popular")
builder.add_text("Machine learning transforms industries")  
builder.add_text("Neural networks process complex data")
builder.add_text("Leann is a great storage saving engine for RAG on your macbook")
builder.build_index("knowledge.leann")
# 2. Search with real-time embeddings
searcher = LeannSearcher("knowledge.leann")
results = searcher.search("programming languages", top_k=2, recompute_beighbor_embeddings=True)
print(results)

llm_config = {"type": "ollama", "model": "qwen3:8b"}

chat = LeannChat(index_path="knowledge.leann", llm_config=llm_config)

response = chat.ask(
    "Compare the two retrieved programming languages and say which one is more popular today. Respond in a single well-formed sentence.",
    top_k=2,
    recompute_beighbor_embeddings=True,
)
print(response)

[leann_backend_hnsw.hnsw_embedding_server LOG]: Received ZMQ request of size 17 bytes
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload length: 1
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload[0]: <class 'str'> - __QUERY_MODEL__
[leann_backend_hnsw.hnsw_embedding_server LOG]: Received ZMQ request of size 38 bytes
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload length: 2
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload[0]: <class 'str'> - __UPDATE_MODEL__
[leann_backend_hnsw.hnsw_embedding_server LOG]: DEBUG: request_payload[1]: <class 'str'> - facebook/contriever
[leann_backend_hnsw.hnsw_embedding_server LOG]: INFO: Updating server model from facebook/contriever-msmarco to: facebook/contriever
[leann_backend_hnsw.hnsw_embedding_server LOG]: INFO: Releasing old model from memory...
[leann_backend_hnsw.hnsw_embedding_server LOG]: Loading new tokenizer for facebook/contriever...
[leann_backend_hnsw.h